In [12]:
# This box deals with importing and defines a few functions.  Do not change.
from __future__ import print_function
import sys

for i in range(len(sys.path))[::-1]:
    if 'antid' in sys.path[i]:
        sys.path.pop(i)
sys.path.insert(0,'/home/ntv/workspace/mantid/release/bin/')
sys.path.insert(1,'/SNS/users/ntv/integrate/jupyter_demo/profile_fitting_jupyter/lib/')
from IPython.display import display
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mantid.simpleapi import *
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import DemoTools
import ICCFitTools as ICCFT
import pickle

def showModels(x):
    plt.figure(num=1,figsize=(10,10));
    plt.clf()
    vMin = 0
    vMax = 0.8*n_events.max()
    plt.subplot(1,2,1)
    plt.imshow(n_events[:,:,x],vmin=vMin,vmax=vMax)
    plt.title('Data')
    plt.subplot(1,2,2)
    plt.imshow(Y3D1[:,:,x],vmin=vMin,vmax=vMax)
    plt.title('Model')


In [9]:
# These are parameters for fitting.

qLow = -5  # Lowest value of q for ConvertToMD 
qHigh = 5; # Highest value of q for ConvertToMD
Q3DFrame='Q_lab' # Either 'Q_lab' or 'Q_sample'; Q_lab recommended if using a strong peaks
                 # profile library from a different sample
eventFileName = '/SNS/MANDI/IPTS-8776/0/5921/NeXus/MANDI_5921_event.nxs' #Full path to the event nexus file
peaksFile = '/SNS/MANDI/shared/ProfileFitting/demo_5921.integrate' #Full path to the ISAW peaks file
UBFile = '/SNS/MANDI/shared/ProfileFitting/demo_5921.mat' #Full path to the ISAW UB file
strongPeakParamsFile = '/SNS/MANDI/shared/ProfileFitting/strongPeakParams_beta_lac_mut_mbvg.pkl' #Full path to pkl file
#strongPeakParamsFile = None
moderatorCoefficientsFile = '/SNS/MANDI/shared/ProfileFitting/franz_coefficients_2017.dat' #Full path to pkl file
IntensityCutoff = 200 # Minimum number of counts to not force a profile
EdgeCutoff = 3 # Pixels within EdgeCutoff from a detector edge will be have a profile forced. Currently for Anger cameras only.
FracStop = 0.05 # Fraction of max counts to include in peak selection.
MinpplFrac = 0.9 # Min fraction of predicted background level to check
MaxpplFrac = 1.1 # Max fraction of predicted background level to check
DQMax = 0.25 # Largest total side length (in Angstrom) to consider for profile fitting.

In [10]:
# Do not edit this cell - only need to run once to load everything
event_ws = Load(Filename=eventFileName, OutputWorkspace='event_ws')
MDdata = ConvertToMD(InputWorkspace='event_ws',  QDimensions='Q3D', dEAnalysisMode='Elastic',
                         Q3DFrames=Q3DFrame, QConversionScales='Q in A^-1',
                         MinValues='%f, %f, %f' % (qLow, qLow, qLow), Maxvalues='%f, %f, %f' % (qHigh, qHigh, qHigh), MaxRecursionDepth=10,
                         LorentzCorrection=False)
peaks_ws = LoadIsawPeaks(Filename=peaksFile, OutputWorkspace='peaks_ws')
DemoTools.addInstrumentParameters(peaks_ws) #Temporary
LoadIsawUB(InputWorkspace=peaks_ws, Filename=UBFile)
UBMatrix = peaks_ws.sample().getOrientedLattice().getUB()
dQ = np.abs(ICCFT.getDQFracHKL(UBMatrix, frac=0.5))
dQ[dQ>DQMax]=DQMax

if strongPeakParamsFile is None:
    strongPeakParams = None
else:
    strongPeakParams = pickle.load(open(strongPeakParamsFile, 'rb'))

padeCoefficients = ICCFT.getModeratorCoefficients(moderatorCoefficientsFile)
NTheta = peaks_ws.getInstrument().getIntParameter("numBinsTheta")[0]
NPhi = peaks_ws.getInstrument().getIntParameter("numBinsPhi")[0]
MindtBinWidth = peaks_ws.getInstrument().getNumberParameter("minDTBinWidth")[0]
MaxdtBinWidth = peaks_ws.getInstrument().getNumberParameter("maxDTBinWidth")[0]
FracHKL = 0.4 # Fraction of HKL to consider for profile fitting.
DQPixel = peaks_ws.getInstrument().getNumberParameter("DQPixel")[0]
np.warnings.filterwarnings('ignore') # There can be a lot of warnings for bad solutions.

In [13]:
reload(DemoTools)
DemoTools.addInstrumentParameters(peaks_ws) #Temporary
peakToFit = 30 # Which peak from PeaksFile to fit.  User should make sure the peak is in the loaded run.
plotSeparateResults = True # True to plot TOF/BVG separately in addition to 3D slices
slider, n_events, Y3D1 = DemoTools.showPeakFit(peakToFit, peaks_ws, MDdata, UBMatrix, dQ, 
                                           padeCoefficients,mindtBinWidth=MindtBinWidth,
                                           dQPixel=DQPixel,fracHKL=FracHKL,pplmin_frac=MinpplFrac,
                                           pplmax_frac=MaxpplFrac, nTheta=NTheta, nPhi=NPhi,intensityCutoff=IntensityCutoff,
                                           edgeCutoff=EdgeCutoff,fracStop=FracStop,plotResults=plotSeparateResults,
                                            strongPeakParams=strongPeakParams)
interact(showModels, x=slider);

Peak 30: New: 238; Ell: 371


interactive(children=(IntSlider(value=19, continuous_update=False, description=u'z Slice:', max=42), Output()), _dom_classes=('widget-interact',))